In [12]:
import fastf1
import pandas as pd

# Get the base directory path
fastf1.Cache.enable_cache('../data')  # Cache in /data

In [ ]:
def get_qualifying_data(year, session_name):
    """
    Get qualifying data for a specific F1 Grand Prix session
    
    Parameters:
    year (int): The year of the season (e.g., 2024, 2023)
    session_name (str): Name of the Grand Prix (e.g., 'Monaco', 'British', 'Abu Dhabi')
    
    Returns:
    pandas.DataFrame: Qualifying results with times in seconds
    """
    
    try:
        # Load the qualifying session
        print(f"Loading {year} {session_name} Grand Prix qualifying data...")
        session = fastf1.get_session(year, session_name, 'Q')
        session.load()
        
        # Get qualifying results
        qualifying_results = session.results
        
        # Create a clean dataframe with relevant information
        df = pd.DataFrame({
            'Position': qualifying_results['Position'],
            'Driver': qualifying_results['Abbreviation'],
            'Full_Name': qualifying_results['FullName'],
            'Team': qualifying_results['TeamName'],
            'Q1_Time_Seconds': qualifying_results['Q1'].dt.total_seconds(),
            'Q2_Time_Seconds': qualifying_results['Q2'].dt.total_seconds(),
            'Q3_Time_Seconds': qualifying_results['Q3'].dt.total_seconds(),
            'Best_Time_Seconds': qualifying_results['Q1'].dt.total_seconds()
        })
        
        # Fill the Best_Time_Seconds with the best qualifying time for each driver
        for idx, row in df.iterrows():
            times = [row['Q1_Time_Seconds'], row['Q2_Time_Seconds'], row['Q3_Time_Seconds']]
            valid_times = [t for t in times if pd.notna(t)]
            if valid_times:
                df.at[idx, 'Best_Time_Seconds'] = min(valid_times)
        
        # Sort by position
        df = df.sort_values('Position').reset_index(drop=True)
        
        print(f"Successfully loaded qualifying data for {year} {session_name} Grand Prix")
        print(f"Total drivers: {len(df)}")
        
        return df
        
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        print("Please check if the year and session name are correct.")
        print("Common session names: 'Bahrain', 'Saudi Arabia', 'Australia', 'Japan', 'China', 'Miami', 'Monaco', 'Spain', 'Canada', 'Austria', 'British', 'Hungary', 'Belgium', 'Netherlands', 'Italy', 'Azerbaijan', 'Singapore', 'United States', 'Mexico', 'Brazil', 'Las Vegas', 'Qatar', 'Abu Dhabi'")
        return None

In [4]:
def display_qualifying_summary(df):
    """
    Display a summary of qualifying results
    """
    if df is not None:
        print("\n=== QUALIFYING RESULTS SUMMARY ===")
        print(f"{'Pos':<4} {'Driver':<4} {'Team':<25} {'Best Time (s)':<12} {'Q1':<10} {'Q2':<10} {'Q3':<10}")
        print("-" * 85)
        
        for _, row in df.head(10).iterrows():  # Show top 10
            pos = int(row['Position']) if pd.notna(row['Position']) else 'DNF'
            driver = row['Driver']
            team = row['Team'][:24]  # Truncate team name if too long
            best_time = f"{row['Best_Time_Seconds']:.3f}" if pd.notna(row['Best_Time_Seconds']) else 'N/A'
            q1 = f"{row['Q1_Time_Seconds']:.3f}" if pd.notna(row['Q1_Time_Seconds']) else 'N/A'
            q2 = f"{row['Q2_Time_Seconds']:.3f}" if pd.notna(row['Q2_Time_Seconds']) else 'N/A'
            q3 = f"{row['Q3_Time_Seconds']:.3f}" if pd.notna(row['Q3_Time_Seconds']) else 'N/A'
            
            print(f"{pos:<4} {driver:<4} {team:<25} {best_time:<12} {q1:<10} {q2:<10} {q3:<10}")

In [5]:
# Additional analysis functions
def get_pole_position_info(df):
    """Get information about pole position"""
    if df is not None and len(df) > 0:
        pole_driver = df.iloc[0]
        print(f"\n=== POLE POSITION ===")
        print(f"Driver: {pole_driver['Full_Name']} ({pole_driver['Driver']})")
        print(f"Team: {pole_driver['Team']}")
        print(f"Time: {pole_driver['Best_Time_Seconds']:.3f} seconds")
        return pole_driver
    return None

def compare_qualifying_times(df, driver1, driver2):
    """Compare qualifying times between two drivers"""
    if df is not None:
        d1 = df[df['Driver'] == driver1]
        d2 = df[df['Driver'] == driver2]
        
        if not d1.empty and not d2.empty:
            d1_time = d1.iloc[0]['Best_Time_Seconds']
            d2_time = d2.iloc[0]['Best_Time_Seconds']
            
            if pd.notna(d1_time) and pd.notna(d2_time):
                diff = abs(d1_time - d2_time)
                faster = driver1 if d1_time < d2_time else driver2
                print(f"\n=== DRIVER COMPARISON ===")
                print(f"{driver1}: {d1_time:.3f}s")
                print(f"{driver2}: {d2_time:.3f}s")
                print(f"Difference: {diff:.3f}s ({faster} is faster)")
            else:
                print("One or both drivers don't have valid times")
        else:
            print("One or both drivers not found in the data")

In [9]:
# Example usage - modify these parameters as needed
YEAR = 2025
SESSION_NAME = 'British Grand Prix'  # Change this to any Grand Prix name

# Get the qualifying data
qualifying_df = get_qualifying_data(YEAR, SESSION_NAME)

# Display the results
if qualifying_df is not None:
    display_qualifying_summary(qualifying_df)
    
    # Display the full dataframe
    print(f"\n=== FULL QUALIFYING DATA ===")
    print(qualifying_df)
    
    # Save to CSV if needed
    filename = f"{YEAR}_{SESSION_NAME}_qualifying.csv"
    # concat '../dat
    qualifying_df.to_csv('../data/' + filename, index=False)
    print(f"\nData saved to: {filename}")

core           INFO 	Loading data for British Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Loading 2025 British Grand Prix Grand Prix qualifying data...


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '81', '4', '63', '44', '16', '12', '87', '14', '10', '55', '22', '6', '23', '31', '30', '5', '18', '27', '43']


Successfully loaded qualifying data for 2025 British Grand Prix Grand Prix
Total drivers: 20

=== QUALIFYING RESULTS SUMMARY ===
Pos  Driver Team                      Best Time (s) Q1         Q2         Q3        
-------------------------------------------------------------------------------------
1    VER  Red Bull Racing           84.892       85.886     85.316     84.892    
2    PIA  McLaren                   84.995       85.963     85.316     84.995    
3    NOR  McLaren                   85.010       86.123     85.231     85.010    
4    RUS  Mercedes                  85.029       86.236     85.637     85.029    
5    HAM  Ferrari                   85.084       86.296     85.084     85.095    
6    LEC  Ferrari                   85.121       86.186     85.133     85.121    
7    ANT  Mercedes                  85.374       86.265     85.620     85.374    
8    BEA  Haas F1 Team              85.471       86.005     85.534     85.471    
9    ALO  Aston Martin              85.593  

In [11]:
# Example additional analysis
if qualifying_df is not None:
    get_pole_position_info(qualifying_df)
    
    # Example comparison (modify driver abbreviations as needed)
    compare_qualifying_times(qualifying_df, 'LEC', 'VER')



=== POLE POSITION ===
Driver: Max Verstappen (VER)
Team: Red Bull Racing
Time: 84.892 seconds

=== DRIVER COMPARISON ===
LEC: 85.121s
VER: 84.892s
Difference: 0.229s (VER is faster)
